In [2]:
import pandas as pd
import xarray as xr
import numpy as np
import scipy as scipy
from toolbar.significance_test import r_test

In [3]:
K_type = xr.open_dataset(r"D:/PyFile/p2/data/Time_type_AverFiltAll0.9%_0.3%_3.nc")

In [15]:
print('90%', r_test(62, 0.10))
print('95%', r_test(62, 0.05))
print('99%', r_test(62, 0.01))

90% 0.21083186294286696
95% 0.2500349005300471
99% 0.3248184473571816


In [45]:
type_1 = K_type.sel(type=1)['K'][:-1]
type_2 = K_type.sel(type=2)['K'][:-1]
type_3 = K_type.sel(type=3)['K'][:-1]

# 计算相关系数
corr_123 = np.corrcoef([type_1, type_2, type_3])
print(corr_123)

[[ 1.         -0.28453486 -0.16799297]
 [-0.28453486  1.          0.05889402]
 [-0.16799297  0.05889402  1.        ]]


In [12]:
# 中下游主导型
type_1 = K_type.sel(type=1)['K']
index_1 = np.load(r"D:\PyFile\p2\data\type1_LN.npy")
index_2 = np.load(r"D:\PyFile\p2\data\type1_P_warm.npy")

corr = np.corrcoef([type_1, index_1, index_2])
print(corr)

# 计算偏相关
corr_type_i1 = (corr[0,1] - corr[0,2] * corr[1,2]) / np.sqrt((1 - corr[0,2]**2) * (1 - corr[1,2]**2))
print('因子1偏相关:',corr_type_i1)
corr_type_i2 = (corr[0,2] - corr[0,1] * corr[1,2]) / np.sqrt((1 - corr[0,1]**2) * (1 - corr[1,2]**2))
print('因子2偏相关:',corr_type_i2)

# 新增回归分析代码
import statsmodels.api as sm

# 构建回归矩阵 (n samples × 2 features)
X = np.column_stack((index_1, index_2))
X = sm.add_constant(X)  # 添加常数项
y = type_1.values

# 执行回归
model = sm.OLS(y, X).fit()

# 输出回归结果
print("\n二元线性回归方程：")
print(f"Y = {model.params[0]:.3f} + {model.params[1]:.3f}*X1 + {model.params[2]:.3f}*X2")
print("\n回归结果摘要：")
print(model.summary())

# 计算标准化回归系数（Beta系数）
beta_coef = model.params[1:] * np.std(X[:,1:], axis=0) / np.std(y)
print("\n标准化回归系数：")
print(f"Beta1(X1): {beta_coef[0]:.3f}, Beta2(X2): {beta_coef[1]:.3f}")

# 在回归结果后添加：
print("\n方差分解:")
print(f"总解释方差 (R²): {model.rsquared:.2%}")
print(f"因子1独特贡献: {beta_coef[0]**2:.2%}")
print(f"因子2独特贡献: {beta_coef[1]**2:.2%}")

[[1.         0.32217064 0.37765848]
 [0.32217064 1.         0.36505605]
 [0.37765848 0.36505605 1.        ]]
因子1偏相关: 0.21379963441213856
因子2偏相关: 0.2950576435313691

二元线性回归方程：
Y = 18.500 + 1.617*X1 + 2.281*X2

回归结果摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     6.555
Date:                Fri, 18 Apr 2025   Prob (F-statistic):            0.00269
Time:                        16:37:38   Log-Likelihood:                -207.52
No. Observations:                  62   AIC:                             421.0
Df Residuals:                      59   BIC:                             427.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                   

In [13]:
# 全域一致型
type_2 = K_type.sel(type=2)['K'][:-1]
index_3 = np.load(r"D:\PyFile\p2\data\type2_In_pre.npy")
index_4 = np.load(r"D:\PyFile\p2\data\type2_LN.npy")

corr = np.corrcoef([type_2, index_3, index_4])
print(corr)

# 计算偏相关
corr_type_i3 = (corr[0,1] - corr[0,2] * corr[1,2]) / np.sqrt((1 - corr[0,2]**2) * (1 - corr[1,2]**2))
print('因子1偏相关:',corr_type_i3)
corr_type_i4 = (corr[0,2] - corr[0,1] * corr[1,2]) / np.sqrt((1 - corr[0,1]**2) * (1 - corr[1,2]**2))
print('因子2偏相关:',corr_type_i4)

# 新增回归分析代码
import statsmodels.api as sm

# 构建回归矩阵 (n samples × 2 features)
X = np.column_stack((index_3, index_4))
X = sm.add_constant(X)  # 添加常数项
y = type_2.values

# 执行回归
model = sm.OLS(y, X).fit()

# 输出回归结果
print("\n二元线性回归方程：")
print(f"Y = {model.params[0]:.3f} + {model.params[1]:.3f}*X1 + {model.params[2]:.3f}*X2")
print("\n回归结果摘要：")
print(model.summary())

# 计算标准化回归系数（Beta系数）
beta_coef = model.params[1:] * np.std(X[:,1:], axis=0) / np.std(y)
print("\n标准化回归系数：")
print(f"Beta1(X1): {beta_coef[0]:.3f}, Beta2(X2): {beta_coef[1]:.3f}")

# 在回归结果后添加：
print("\n方差分解:")
print(f"总解释方差 (R²): {model.rsquared:.2%}")
print(f"因子1独特贡献: {beta_coef[0]**2:.2%}")
print(f"因子2独特贡献: {beta_coef[1]**2:.2%}")

[[1.         0.31606781 0.38520242]
 [0.31606781 1.         0.29138464]
 [0.38520242 0.29138464 1.        ]]
因子1偏相关: 0.2308890014553651
因子2偏相关: 0.32295696053750794

二元线性回归方程：
Y = 12.410 + 1.859*X1 + 2.674*X2

回归结果摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.194
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     6.970
Date:                Sun, 20 Apr 2025   Prob (F-statistic):            0.00194
Time:                        22:40:46   Log-Likelihood:                -209.43
No. Observations:                  61   AIC:                             424.9
Df Residuals:                      58   BIC:                             431.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                   

In [49]:
# 上游主导型
type_3 = K_type.sel(type=3)['K']
index_5 = np.load(r"D:\PyFile\p2\data\type3_EL.npy")
index_6 = np.load(r"D:\PyFile\p2\data\type3_P_cold.npy")

corr = np.corrcoef([type_3, index_5, index_6])
print(corr)
# 计算偏相关
corr_type_i5 = (corr[0,1] - corr[0,2] * corr[1,2]) / np.sqrt((1 - corr[0,2]**2) * (1 - corr[1,2]**2))
print('因子1偏相关:',corr_type_i5)
corr_type_i6 = (corr[0,2] - corr[0,1] * corr[1,2]) / np.sqrt((1 - corr[0,1]**2) * (1 - corr[1,2]**2))
print('因子2偏相关:',corr_type_i6)


# 新增回归分析代码
import statsmodels.api as sm

# 构建回归矩阵 (n samples × 2 features)
X = np.column_stack((index_5, index_6))
X = sm.add_constant(X)  # 添加常数项
y = type_3.values

# 执行回归
model = sm.OLS(y, X).fit()

# 输出回归结果
print("\n二元线性回归方程：")
print(f"Y = {model.params[0]:.3f} + {model.params[1]:.3f}*X1 + {model.params[2]:.3f}*X2")
print("\n回归结果摘要：")
print(model.summary())

# 计算标准化回归系数（Beta系数）
beta_coef = model.params[1:] * np.std(X[:,1:], axis=0) / np.std(y)
print("\n标准化回归系数：")
print(f"Beta1(X1): {beta_coef[0]:.3f}, Beta2(X2): {beta_coef[1]:.3f}")

# 在回归结果后添加：
print("\n方差分解:")
print(f"总解释方差 (R²): {model.rsquared:.2%}")
print(f"因子1独特贡献: {beta_coef[0]**2:.2%}")
print(f"因子2独特贡献: {beta_coef[1]**2:.2%}")

[[1.         0.2398418  0.3355742 ]
 [0.2398418  1.         0.33584391]
 [0.3355742  0.33584391 1.        ]]
因子1偏相关: 0.14329013033785581
因子2偏相关: 0.27889089109353593

二元线性回归方程：
Y = 4.177 + 0.688*X1 + 1.380*X2

回归结果摘要：
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     4.440
Date:                Wed, 16 Apr 2025   Prob (F-statistic):             0.0160
Time:                        15:29:01   Log-Likelihood:                -180.89
No. Observations:                  62   AIC:                             367.8
Df Residuals:                      59   BIC:                             374.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                   